## Using XGBoost in pipelines

In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np

### Encoding categorical columns I: LabelEncoder

In [4]:
df = pd.read_csv('chapter4-data.csv')

In [5]:

from sklearn.preprocessing import LabelEncoder


df.LotFrontage = df.LotFrontage.fillna(0)


categorical_mask = (df.dtypes == object)


categorical_columns = df.columns[categorical_mask].tolist()


print(df[categorical_columns].head())


le = LabelEncoder()


df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))


print(df[categorical_columns].head())

  MSZoning PavedDrive Neighborhood BldgType HouseStyle
0       RL          Y      CollgCr     1Fam     2Story
1       RL          Y      Veenker     1Fam     1Story
2       RL          Y      CollgCr     1Fam     2Story
3       RL          Y      Crawfor     1Fam     2Story
4       RL          Y      NoRidge     1Fam     2Story
   MSZoning  PavedDrive  Neighborhood  BldgType  HouseStyle
0         3           2             5         0           5
1         3           2            24         0           2
2         3           2             5         0           5
3         3           2             6         0           5
4         3           2            15         0           5


### Encoding categorical columns II: OneHotEncoder

In [6]:

from sklearn.preprocessing import OneHotEncoder


ohe = OneHotEncoder(sparse=False)


df_encoded = ohe.fit_transform(df)


print(df_encoded[:5, :])


print(df.shape)


print(df_encoded.shape)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1460, 22)
(1460, 4829)
